# PDF Loader:
The success of the KPI extractor will depend in part on our ability to read in pdf data without data corruption. Below we test various approaches for loading pdfs. 

In [1]:
# Move to root directory
import os

notebooks_dir = 'notebooks'
if notebooks_dir in os.path.abspath(os.curdir):
    while not os.path.abspath(os.curdir).endswith('notebooks'):
        print(os.path.abspath(os.curdir))
        os.chdir('..')
    os.chdir('..')  # to get to root

print(os.path.abspath(os.curdir))

C:\Users\MD726YR\PycharmProjects\eyalytics


In [2]:
# Supress SSL verification (EY problem):
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning

# Suppress the warning from urllib3.
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

old_send = requests.Session.send

def new_send(*args, **kwargs):
    kwargs['verify'] = False
    return old_send(*args, **kwargs)

requests.Session.send = new_send

In [3]:
import re
import camelot
import tabula
import pandas as pd

from itertools import groupby
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.high_level import extract_pages
from io import BytesIO, StringIO
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text, extract_text_to_fp

from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTImage, LTFigure
# from pdfminer.high_level import get_layouts  # no longer available
from pdfminer.pdfpage import PDFPage

from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

from prettytable import PrettyTable

from pandas.testing import assert_frame_equal

## BASELINE:
Existing simplest solution:

In [4]:
# Create URL loader:
def get_pdf_content_from_url(url: str, page_start: int = 0, n_pages: int = None) -> str:
    try:
        # Fetch the PDF content using requests
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response returned an unsuccessful status code

        # If n_pages is not specified, read till the end
        end_page = page_start + n_pages if n_pages is not None else None

        # Use BytesIO to convert the byte stream to a file-like object so it can be read by pdfminer
        with BytesIO(response.content) as pdf_data, StringIO() as output_string:
            # Extract text from the PDF for the specified pages
            extract_text_to_fp(
                pdf_data, output_string, laparams=LAParams(), 
                page_numbers=list(range(page_start, end_page or int(1e9)))
            )  # use a large number as default if end_page is None
            return output_string.getvalue()
    except requests.RequestException as e:
        print(f"Error fetching the PDF: {e}")
        return ""
    except Exception as e:
        print(f"Error processing the PDF: {e}")
        return ""

# Testing the function
url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-and-sustainability-report-2022.pdf"
text = get_pdf_content_from_url(url, page_start=17, n_pages=1)

In [5]:
print(text)

CONTENTS

CEO MESSAGE

EXECUTIVE SUMMARY

OUR COMPANY

WATER

PORTFOLIO

PACKAGING

CLIMATE

AGRICULTURE

PEOPLE

OPERATIONS

DATA APPENDIX

FRAMEWORKS

OVERVIEW

AT A GLANCE

HOW WE OPERATE

INNOVATION

FINANCIAL HIGHLIGHTS

GOVERNANCE & MANAGEMENT

PRIORITY TOPICS

STAKEHOLDER ENGAGEMENT & PARTNERSHIPS

E
T
A
R
E
P
O
E
W
W
O
H

The Coca-Cola Company markets, 
manufactures and sells:

•  Beverage concentrates and syrups

•    Finished beverages (including sparkling soft 

drinks; water, sports, coffee & tea; juice, value-

added dairy & plant-based beverages; and 

emerging beverages).

In our concentrate operations, The Coca-Cola 

Company typically generates net operating 

revenues by selling concentrates and syrups  

to authorized bottling partners.

Our bottling partners combine the concentrates 

and syrups with still or sparkling water  

and sweeteners (depending on the product),  

to prepare, package, distribute and sell  

finished beverages.

Our finished product operatio

## Attempt 1:
Load in textual information seperately from tabular information.

#### Notes:

**Loading Text:**
To get more structured results, you can dive a bit deeper into the pdfminer library to analyze the individual layout components of the PDF. By iterating through these components, you can customize your extraction strategy based on the type and structure of each component.

Here's a more granular approach using pdfminer:

- Parse the PDF and extract layout objects.
- Classify each object based on its type (text box, image, line, etc.)
- Custom process each text box, ensuring tables (previously identified) are skipped to avoid duplicate extraction.

**Loading Table:**
This automatic detection sometimes confuses text layouts with tables, especially when the text is arranged in columns or has other grid-like structures.

To refine the extraction process, we can use a combination of heuristic approaches:

- Text Density Check: For each table detected by camelot, calculate the text density. If there's a lot of whitespace relative to text, it might not be a table.

- Row and Column Count: Check the number of rows and columns. If there's just one column or one row, it might be just regular text instead of a table.

- Row Uniformity: Check if the rows have similar content. If each row is wildly different in terms of content length or type, it might not be a table.

**Improvements**:
Few additional ideas and improvements could help capture more structured content:

*Context Preservation around Tables:*
As you mentioned, it might be valuable to add sentences or headings appearing immediately before and after tables as added context.You can modify the extract_text_custom function to look for text boxes immediately above and below identified table regions and append these to the table markdown content.

*Images and Figures Extraction:*
Right now, the images or figures are skipped. Depending on the nature of the PDFs you're working with, images, charts, and figures might contain essential information. Tools like Pytesseract can be used to OCR images, but the quality of results can vary.

*Use PyPDF2 for Meta Information:*
Sometimes the meta information of a PDF, such as the title, author, or other metadata, contains important context about the content. You can extract this with PyPDF2 and include it in your results.

*Error Handling for Individual Pages:*
Instead of failing the entire operation when an error occurs on one page, you can handle errors on a per-page basis. This way, even if one page causes an issue, the rest of the pages can still be processed.

*Text Post-processing:*
Once the text is extracted, you might notice inconsistencies like broken sentences or words. Depending on the quality and structure of the PDFs, some text post-processing might be helpful. This could involve:
- Removing any residual special characters or artifacts.
- Reconstructing broken sentences.
- Removing footers, headers, or repeated elements.

*Parallel Processing:*
If the processing time becomes an issue, especially with large PDFs, you can consider parallel processing for different pages. This would involve extracting content from multiple pages concurrently. Python’s multiprocessing module can help here.

In [46]:
def is_valid_table(table):
    df = table.df
    
    # 1. Text Density Check
    text_density = sum(len(cell) for row in df.values for cell in row) / (df.shape[0] * df.shape[1])
    if text_density < 3:  # This threshold might need tweaking
        return False

    # 2. Row and Column Count
    if df.shape[0] == 1 or df.shape[1] == 1:
        return False

    # 3. Row Uniformity (check variation in the length of content in each row)
    row_lengths = [sum(len(cell) for cell in row) for row in df.values]
    length_variation = max(row_lengths) - min(row_lengths)
    if length_variation > 80:  # Again, you might need to tweak this
        return False

    return True


def get_context_around_table(layout, table_region, distance=30):
    context_above, context_below = "", ""
    for obj in layout:
        if isinstance(obj, LTTextBox):
            # Check if the text box is immediately above the table
            if (
                obj.bbox[2] > table_region[0] and obj.bbox[0] < table_region[2] and
                abs(obj.bbox[1] - table_region[3]) < distance
            ):
                context_above += obj.get_text()

            # Check if the text box is immediately below the table
            if (obj.bbox[2] > table_region[0] and obj.bbox[0] < table_region[2] and
                abs(obj.bbox[3] - table_region[1]) < distance):
                context_below += obj.get_text()

    return context_above, context_below


def clean_text(text):
    # replace common artifacts with appropriate characters or remove them
    cleaned = text.replace("•", "-")
    cleaned = cleaned.replace("\x00", "")
    return cleaned


def extract_text_custom(layout, table_regions):
    extracted_text = ""
    
    for obj in layout:
        if isinstance(obj, LTTextBox) or isinstance(obj, LTTextLine):
            skip = False
            for region in table_regions:
                # Skip text boxes that overlap with identified table regions
                if (obj.bbox[0] <= region[2] and obj.bbox[2] >= region[0] and
                    obj.bbox[1] <= region[3] and obj.bbox[3] >= region[1]):
                    skip = True
                    break
            if not skip:
                text = obj.get_text()
                normalized_text = ' '.join(text.split())
                normalized_text = clean_text(normalized_text)
                
                # Check if it's a heading (for demonstration; adjust accordingly)
                if hasattr(obj, "size") and obj.size > 20:  # SOME_THRESHOLD:
                    extracted_text += "\n[HEADING]" + normalized_text + "[/HEADING]\n"
                else:
                    extracted_text += "[START_BLOCK]" + normalized_text + "[END_BLOCK]\n"
        # Optionally handle other types of objects, like images or figures if required.
        # For now, skipping them.
    
    return extracted_text


# Modifying table detection to catch more tables
def detect_table_regions_for_page(page_layout):
    table_regions = []

    for element in page_layout:
        if isinstance(element, LTFigure):
            bbox = element.bbox
            table_regions.append(bbox)
            
    return table_regions


def extract_content_from_pdf(pdf_path, start_page=None, end_page=None):
    content = ""
    
    page_number = start_page
    for page_layout in extract_pages(pdf_path, laparams=LAParams(), page_numbers=range(start_page-1, end_page)):
        # Extract table regions for the page
        table_regions_for_page = detect_table_regions_for_page(page_layout)
        
        # Convert table regions to format accepted by Camelot 
        table_areas = ["{},{},{},{}".format(*region) for region in table_regions_for_page]
        
        # 1. Attempt Custom Table Extraction
        try:
            # Attempt to extract tables using the custom regions
            tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream', table_areas=table_areas)
            
            for table in tables:
                if not is_valid_table(table):
                    continue
                    
                # Get context around the table and append to content
                context_above, context_below = get_context_around_table(page_layout, table._bbox)
                content += context_above
                content += table.df.to_string(index=False, header=False)
                content += context_below
                content += "\n\n"
                
            # Extract any remaining text, ensuring we don't double-extract
            content += extract_text_custom(page_layout, table_regions_for_page)
            
        except Exception as e:
            print(f"Error with custom table extraction on page {page_number}: {e}")
            
            # 2. Fallback to Default Table Extraction
            try:
                tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream')
                for table in tables:
                    content += table.df.to_string(index=False, header=False)
                    content += "\n\n"
                
                # Extract any remaining text, ensuring we don't double-extract
                content += extract_text_custom(page_layout, table_regions_for_page)
                
            except Exception as e2:
                print(f"Error with default table extraction on page {page_number}: {e2}")
                
                # 3. If All Else Fails, Extract Textual Information
                content += extract_text_custom(page_layout, [])
        
        # Increment the page_number
        page_number += 1

    return content


def get_pdf_content_from_url_3(url, page_start=1, n_pages=None):
    try:
        response = requests.get(url)
        response.raise_for_status()

        with BytesIO(response.content) as pdf_data:
            # Save to a temporary file as some libraries need a filepath
            temp_pdf_path = "./data/pdf_db/temp_pdf_file.pdf"
            with open(temp_pdf_path, 'wb') as temp_file:
                temp_file.write(pdf_data.read())

            # If n_pages is specified, set the end page. Otherwise, process till the last page.
            end_page = page_start + n_pages - 1 if n_pages else None

            content = extract_content_from_pdf(
                temp_pdf_path, start_page=page_start, end_page=end_page
            )
            return content

    except requests.RequestException as e:
        print(f"Error fetching the PDF: {e}")
        return ""
    except Exception as e:
        print(f"Error processing the PDF: {e}")
        return ""
    
text = get_pdf_content_from_url_3(url, page_start=17, n_pages=1)

In [47]:
print(text)

[START_BLOCK]CONTENTS[END_BLOCK]
[START_BLOCK]CEO MESSAGE[END_BLOCK]
[START_BLOCK]EXECUTIVE SUMMARY[END_BLOCK]
[START_BLOCK]OUR COMPANY[END_BLOCK]
[START_BLOCK]WATER[END_BLOCK]
[START_BLOCK]PORTFOLIO[END_BLOCK]
[START_BLOCK]PACKAGING[END_BLOCK]
[START_BLOCK]CLIMATE[END_BLOCK]
[START_BLOCK]AGRICULTURE[END_BLOCK]
[START_BLOCK]PEOPLE[END_BLOCK]
[START_BLOCK]OPERATIONS[END_BLOCK]
[START_BLOCK]DATA APPENDIX[END_BLOCK]
[START_BLOCK]FRAMEWORKS[END_BLOCK]
[START_BLOCK]OVERVIEW[END_BLOCK]
[START_BLOCK]AT A GLANCE[END_BLOCK]
[START_BLOCK]HOW WE OPERATE[END_BLOCK]
[START_BLOCK]INNOVATION[END_BLOCK]
[START_BLOCK]FINANCIAL HIGHLIGHTS[END_BLOCK]
[START_BLOCK]GOVERNANCE & MANAGEMENT[END_BLOCK]
[START_BLOCK]PRIORITY TOPICS[END_BLOCK]
[START_BLOCK]STAKEHOLDER ENGAGEMENT & PARTNERSHIPS[END_BLOCK]
[START_BLOCK]S T H G I L H G I H L A I C N A N I F[END_BLOCK]
[START_BLOCK](in millions except per share data)[END_BLOCK]
[START_BLOCK]2019[END_BLOCK]
[START_BLOCK]2020[END_BLOCK]
[START_BLOCK]2021[END_BLOCK]
[

### Table processing:
Attempt to add functionality for processing textual information to .md format: specifically tables which the program fails to load.

In [3]:
import re

def extract_blocks(text):
    """Extracts blocks from the text."""
    return re.findall(r'\[START_BLOCK\](.*?)\[END_BLOCK\]', text, re.DOTALL)

def is_value(block):
    """Checks if a block represents a numeric or monetary value."""
    return bool(re.search(r'\$?[\d,]+\.?\d{0,2}|\d+%?', block))

def extract_columns_from_blocks(blocks):
    """Segregates blocks into potential columns based on value type."""
    columns = []
    current_column = []

    i = 0
    while i < len(blocks):
        block = blocks[i]
        if not is_value(block):
            current_column = [block]
            i += 1
            while i < len(blocks) and is_value(blocks[i]):
                current_column.append(blocks[i])
                i += 1
            columns.append(current_column)
        else:
            i += 1

    return columns

def columns_to_md_table(columns):
    """Converts list of columns to markdown table."""
    # Convert columns to rows
    max_length = max(len(column) for column in columns)
    rows = []

    for i in range(max_length):
        row = []
        for column in columns:
            row.append(column[i] if i < len(column) else "")
        rows.append(row)

    # Construct markdown table
    table = '| ' + ' | '.join(col[0] for col in columns) + ' |\n'
    table += '| ' + ' | '.join(['---'] * len(columns)) + ' |\n'
    
    for row in rows[1:]:
        table += '| ' + ' | '.join(row) + ' |\n'
    
    return table

def extract_md_table(text):
    blocks = extract_blocks(text)
    columns = extract_columns_from_blocks(blocks)
    return columns_to_md_table(columns)

# Test the function
text = """
[START_BLOCK](in millions except per share data)[END_BLOCK]
[START_BLOCK]2019[END_BLOCK]
[START_BLOCK]2020[END_BLOCK]
[START_BLOCK]2021[END_BLOCK]
[START_BLOCK]2022[END_BLOCK]
[START_BLOCK]Summary of Operations[END_BLOCK]
[START_BLOCK]Net operating revenues[END_BLOCK]
[START_BLOCK]$37,266[END_BLOCK]
[START_BLOCK]$33,014[END_BLOCK]
[START_BLOCK]$38,655[END_BLOCK]
[START_BLOCK]$ 43,004[END_BLOCK]
[START_BLOCK]Operating income[END_BLOCK]
[START_BLOCK]10,086[END_BLOCK]
[START_BLOCK]8,997[END_BLOCK]
[START_BLOCK]10,308[END_BLOCK]
[START_BLOCK]10,909[END_BLOCK]
"""

print(extract_md_table(text))


| (in millions except per share data) | Summary of Operations | Net operating revenues | Operating income |
| --- | --- | --- | --- |
| 2019 |  | $37,266 | 10,086 |
| 2020 |  | $33,014 | 8,997 |
| 2021 |  | $38,655 | 10,308 |
| 2022 |  | $ 43,004 | 10,909 |



**Developing a working version of this that would be generic enough to work across multiple files would be too complex, hence avoid further development of this solution.**

### Attempt 2: 

Attempt to use selenium and a web-browser to read a pdf file:

In [34]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import time
import pyperclip

# Setup for Firefox
firefox_options = webdriver.FirefoxOptions()
# If there are specific options you want to set for Firefox, add them here

driver_path = "./drivers/geckodriver.exe"
service = Service(driver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)

driver.get("https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-and-sustainability-report-2022.pdf")
time.sleep(5)  # Waiting for the PDF to load

# Number of pages to scrape
start_page = 17
end_page = 20  # For example, you can set this to the last page you want to scrape

# Store the accumulated content
accumulated_content = ""

# Loop through the desired range of pages
for page_number in range(start_page, end_page + 1):
    # Navigating to the specific page using the 'Next' button
    next_button = driver.find_element(By.CSS_SELECTOR, '#next')
    while True:
        current_page = int(driver.find_element(By.CSS_SELECTOR, 'input#pageNumber').get_attribute("value"))
        if current_page == page_number:
            break
        next_button.click()
        time.sleep(1)  # Wait for the page to load

    # Using JavaScript to select all content within the specific textLayer for the current page
    js_script = """
    var range = document.createRange();
    var content = document.querySelector('div.page:nth-child({}) > div:nth-child(2)');
    range.selectNodeContents(content);
    window.getSelection().addRange(range);
    return true;
    """.format(page_number)
    success = driver.execute_script(js_script)

    if success:
        # Use JavaScript to copy the selected content
        driver.execute_script("document.execCommand('copy');")
        time.sleep(2)  # Waiting for the clipboard to populate
        accumulated_content += pyperclip.paste()  # Accumulate the content

# Print accumulated content
print(accumulated_content)

driver.quit()


17THE COCA-COLA COMPANY 2022 BUSINESS & SUSTAINABILITY REPORT
HOW WE OPERATEAT A GLANCEOVERVIEW INNOVATION FINANCIAL HIGHLIGHTS GOVERNANCE & MANAGEMENT PRIORITY TOPICS STAKEHOLDER ENGAGEMENT & PARTNERSHIPS
CEO MESSAGE EXECUTIVE SUMMARYCONTENTS AGRICULTUREPORTFOLIO DATA APPENDIXPACKAGING FRAMEWORKSWATER OPERATIONSCLIMATE PEOPLEOUR COMPANY
FINANCIAL HIGHLIGHTS
Note: See pages 72–74 for reconciliations of non-GAAP financial measures to our results as reported under accounting principles generally accepted
in the United States (U.S. GAAP).
1 Reported net operating revenues grew 9%, declined 11%, grew 17%
and grew 11% for the years ended December 31, 2019, 2020, 2021
and 2022, respectively.
2 Reported operating income grew 10%, declined 11%, grew 15% and
grew 6% for the years ended December 31, 2019, 2020, 2021 and
2022, respectively.
3 Reported diluted earnings per share grew 38%, declined 13%, grew
26% and declined 3% for the years ended December 31, 2019, 2020,
2021 and 2022, respectivel

Still unsure if the above is the way to go.

### Attempt 3:
add tabula loader to attempt 1. 

In [7]:
def is_valid_table(table):
    df = table.df

    # 1. Text Density Check
    text_density = sum(len(cell) for row in df.values for cell in row) / (df.shape[0] * df.shape[1])
    if text_density < 3:  # This threshold might need tweaking
        return False

    # 2. Row and Column Count
    if df.shape[0] == 1 or df.shape[1] == 1:
        return False

    # 3. Row Uniformity
    row_lengths = [sum(len(cell) for cell in row) for row in df.values]
    length_variation = max(row_lengths) - min(row_lengths)
    if length_variation > 80:  # Again, tweak if necessary
        return False

    return True


def get_context_around_table(layout, table_region, distance=30):
    context_above, context_below = "", ""
    for obj in layout:
        if isinstance(obj, LTTextBox):
            # Check if the text box is immediately above the table
            if (
                obj.bbox[2] > table_region[0] and obj.bbox[0] < table_region[2] and
                abs(obj.bbox[1] - table_region[3]) < distance
            ):
                context_above += obj.get_text()

            # Check if the text box is immediately below the table
            if (obj.bbox[2] > table_region[0] and obj.bbox[0] < table_region[2] and
                abs(obj.bbox[3] - table_region[1]) < distance):
                context_below += obj.get_text()

    return context_above, context_below


def clean_text(text):
    # replace common artifacts with appropriate characters or remove them
    cleaned = text.replace("•", "-")
    cleaned = cleaned.replace("\x00", "")
    return cleaned


def extract_text_custom(layout, table_regions):
    extracted_text = ""
    
    for obj in layout:
        if isinstance(obj, LTTextBox) or isinstance(obj, LTTextLine):
            skip = False
            for region in table_regions:
                # Skip text boxes that overlap with identified table regions
                if (obj.bbox[0] <= region[2] and obj.bbox[2] >= region[0] and
                    obj.bbox[1] <= region[3] and obj.bbox[3] >= region[1]):
                    skip = True
                    break
            if not skip:
                text = obj.get_text()
                normalized_text = ' '.join(text.split())
                normalized_text = clean_text(normalized_text)
                
                # Check if it's a heading (for demonstration; adjust accordingly)
                if hasattr(obj, "size") and obj.size > 20:  # SOME_THRESHOLD:
                    extracted_text += "\n[HEADING]" + normalized_text + "[/HEADING]\n"
                else:
                    extracted_text += "[START_BLOCK]" + normalized_text + "[END_BLOCK]\n"
        # Optionally handle other types of objects, like images or figures if required.
        # For now, skipping them.
    
    return extracted_text


def detect_table_regions_for_page(page_layout):
    table_regions = []

    for element in page_layout:
        if isinstance(element, LTFigure):
            bbox = element.bbox
            table_regions.append(bbox)
    return table_regions


def extract_table_with_tabula(pdf_path, page_number):
    tables = tabula.read_pdf(pdf_path, pages=page_number, multiple_tables=True)
    content = ""
    for table in tables:
        print('\n\n')
        print(table)
        print('\n\n')
        content += table.to_string(index=False, header=False)
        content += "\n\n"
    return content


def extract_with_method1(pdf_path, page_number, page_layout, table_regions_for_page):
    table_areas = ["{},{},{},{}".format(*region) for region in table_regions_for_page]
    tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream', table_areas=table_areas)
    
    content = ""
    for table in tables:
        if not is_valid_table(table):
            continue
        context_above, context_below = get_context_around_table(page_layout, table._bbox)
        content += context_above
        content += table.df.to_string(index=False, header=True)
        content += context_below
        content += "\n\n"
    content += extract_text_custom(page_layout, table_regions_for_page)
    
    return content, len(tables)


def extract_with_method2(pdf_path, page_number, page_layout, table_regions_for_page):
    tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream')
    
    content = ""
    for table in tables:
        content += table.df.to_string(index=False, header=True)
        content += "\n\n"
    content += extract_text_custom(page_layout, table_regions_for_page)
    
    return content, len(tables)


def extract_with_method3(pdf_path, page_number, page_layout):
    content = extract_table_with_tabula(pdf_path, page_number)
    content += extract_text_custom(page_layout, [])
    return content


def extract_content_from_pdf(pdf_path, start_page=None, end_page=None):
    content = ""
    page_number = start_page

    for page_layout in extract_pages(pdf_path, laparams=LAParams(), page_numbers=range(start_page-1, end_page)):
        table_regions_for_page = detect_table_regions_for_page(page_layout)
        
        try:
            extracted_content, table_count = extract_with_method1(pdf_path, page_number, page_layout, table_regions_for_page)
            if table_count == 0:
                raise ValueError("No tables found with method 1.")
            content += extracted_content
            
        except Exception as e:
            print(f"Error or no tables found with custom table extraction on page {page_number}: {e}")

            try:
                extracted_content, table_count = extract_with_method2(pdf_path, page_number, page_layout, table_regions_for_page)
                if table_count == 0:
                    raise ValueError("No tables found with method 2.")
                content += extracted_content
                
            except Exception as e2:
                print(f"Error or no tables found with default table extraction on page {page_number}: {e2}")

                try:
                    extracted_content = extract_with_method3(pdf_path, page_number, page_layout)
                    content += extracted_content
                except Exception as e3:
                    print(f"Error with tabula extraction on page {page_number}: {e3}")
                    content += extract_text_custom(page_layout, [])
        page_number += 1

    return content


def get_pdf_content_from_url_3(url, page_start=1, n_pages=None):
    try:
        response = requests.get(url)
        response.raise_for_status()

        with BytesIO(response.content) as pdf_data:
            temp_pdf_path = "./data/pdf_db/temp_pdf_file.pdf"
            with open(temp_pdf_path, 'wb') as temp_file:
                temp_file.write(pdf_data.read())

            end_page = page_start + n_pages - 1 if n_pages else None
            content = extract_content_from_pdf(temp_pdf_path, start_page=page_start, end_page=end_page)
            return content

    except requests.RequestException as e:
        print(f"Error fetching the PDF: {e}")
        return ""
    except Exception as e:
        print(f"Error processing the PDF: {e}")
        return ""
    
url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-and-sustainability-report-2022.pdf"
text = get_pdf_content_from_url_3(url, page_start=17, n_pages=1)
print(text)

Error or no tables found with custom table extraction on page 17: No tables found with method 1.
                                     0       1       2       3        4                                     5                                        6
                                                                                       Organic Revenue Growth    Comparable Currency Neutral Operating
                                                                                                  (Non-GAAP)1                Income Growth (Non-GAAP)2
                                          2019    2020    2021     2022                                                                               
   (in millions except per share data)                                                                                                                
                                                                                                           20                                       

# Attempt 4:
simplify attempt 3. Focusing on what worked and removing bits that didn't work.

In [17]:
def post_process_dataframe(df):
    """
    Post-processes the extracted dataframe to handle common anomalies:
    - Removes ghost columns.
    - Removes ghost rows.
    - Attempts to correct rows with misplaced cells.
    """
    # Remove columns that are mostly empty (ghost columns)
    column_non_empty_ratios = df.count() / len(df)
    threshold_col = 0.15  # Adjust: columns with less than 15% non-empty cells will be dropped
    columns_to_drop = column_non_empty_ratios[column_non_empty_ratios < threshold_col].index
    df = df.drop(columns=columns_to_drop)

    # Remove rows that are mostly empty (ghost rows)
    row_non_empty_ratios = df.apply(lambda row: row.count(), axis=1) / len(df.columns)
    threshold_row = 0.15  # Adjust: rows with less than 15% non-empty cells will be dropped
    df = df[row_non_empty_ratios > threshold_row]

    # Potential for further refinement:
    # - Handle misplaced cells by checking content patterns.
    # - Merge cells that might have been split across rows/columns.

    return df

def extract_tables_with_custom_areas(pdf_path, page_number, page_layout):
    table_areas = ["{},{},{},{}".format(*region) for region in detect_table_regions_for_page(page_layout)]
    tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream', table_areas=table_areas, 
                             split_text=True, strip_text=' .\n', flag_size=True)

    valid_tables = [post_process_dataframe(table.df) for table in tables if is_valid_table(table)]
    return valid_tables

def extract_tables_default(pdf_path, page_number):
    tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream', 
                             split_text=True, strip_text=' .\n', flag_size=True)

    valid_tables = [post_process_dataframe(table.df) for table in tables if is_valid_table(table)]
    return valid_tables

def extract_tables_with_tabula(pdf_path, page_number):
    tables = tabula.read_pdf(pdf_path, pages=page_number, multiple_tables=True)
    
    # We apply post-processing to each table
    processed_tables = [post_process_dataframe(table) for table in tables]
    valid_tables = [table for table in processed_tables if not table.empty]  # Filter out any tables that became empty after processing

    return valid_tables

def extract_tables_from_page(pdf_path, page_number, page_layout):
    methods = [
        lambda: extract_tables_with_custom_areas(pdf_path, page_number, page_layout),
        lambda: extract_tables_default(pdf_path, page_number),
        lambda: extract_tables_with_tabula(pdf_path, page_number)
    ]

    for method in methods:
        try:
            tables = method()
            if tables:
                return tables
        except Exception as e:
            print(f"Error during table extraction with method {methods.index(method) + 1} on page {page_number}: {e}")
    return []

def should_skip(text_obj, table_regions):
    for region in table_regions:
        if (text_obj.x0 >= region['x0'] and text_obj.x1 <= region['x1'] and
            text_obj.y0 >= region['y0'] and text_obj.y1 <= region['y1']):
            return True
    return False

def clean_text(text):
    # replace common artifacts with appropriate characters or remove them
    cleaned = text.replace("•", "-")
    cleaned = cleaned.replace("\x00", "")
    return cleaned

def classify_text_based_on_style(obj):
    """
    Classify text object based on font size and other attributes.
    For now, using a simple size heuristic. This might need further refinement.
    """
    if hasattr(obj, "size"):
        if obj.size > 26:  # Threshold for headers
            return "header"
        elif 20 < obj.size <= 26:  # Threshold for sub-headers or titles
            return "sub-header"
        else:
            return "paragraph"
    return "unknown"

def classify_text_based_on_content(obj):
    """Classify text object based on content heuristics."""
    text = obj.get_text()
    
    # Heuristic checks for footnotes, typically found at the bottom of pages and smaller in size.
    if text.startswith(("*", "1", "2", "3")) and obj.y0 < 200:  # 'obj.y0 < 200' checks if the text is near the bottom
        return "footnote"
    
    # Check for high frequency of numbers; could be table or graph data.
    num_count = sum([1 for char in text if char.isdigit()])
    if num_count > len(text) * 0.5:  # More than 50% numbers
        return "table_or_graph"
    
    # Check for numbers (including potential currency signs, commas, and decimals)
    number_pattern = re.compile(r'^[-£$€]?([\d,]+(\.\d{1,2})?\%?\s*)+$')
    
    if number_pattern.match(text):
        return "number"
    
    return None

def extract_refined_text(layout, table_regions):
    extracted_text = ""
    prev_tag = None

    for obj in layout:
        if isinstance(obj, LTTextBox) or isinstance(obj, LTTextLine):
            skip = False
            for region in table_regions:
                if (obj.bbox[0] <= region[2] and obj.bbox[2] >= region[0] and
                    obj.bbox[1] <= region[3] and obj.bbox[3] >= region[1]):
                    skip = True
                    break
            
            if not skip:
                text = clean_text(obj.get_text())
                text_type_style = classify_text_based_on_style(obj)
                text_type_content = classify_text_based_on_content(obj)

                if text_type_content == "footnote":
                    if prev_tag == "FOOTNOTE":
                        extracted_text = extracted_text.rstrip("[/FOOTNOTE]\n") + " "
                    else:
                        extracted_text += "[FOOTNOTE]"
                    extracted_text += text + "[/FOOTNOTE]\n"
                    prev_tag = "FOOTNOTE"
                elif text_type_content == "table_or_graph":
                    if prev_tag == "DATA":
                        extracted_text = extracted_text.rstrip("[/DATA]\n") + "\n"
                    else:
                        extracted_text += "[DATA]"
                    extracted_text += text + "[/DATA]\n"
                    prev_tag = "DATA"    
                elif text_type_content == "number":
                    if prev_tag == "NUMBER":
                        extracted_text = extracted_text.rstrip("[/NUMBER]\n") + " "
                    else:
                        extracted_text += "[NUMBER]"
                    extracted_text += text + "[/NUMBER]\n"
                    prev_tag = "NUMBER"
                elif text_type_style == "header":
                    extracted_text += "[HEADER]" + text + "[/HEADER]\n\n"
                elif text_type_style == "sub-header":
                    if prev_tag == "SUB-HEADER":
                        extracted_text = extracted_text.rstrip("[/SUB-HEADER]\n") + " "
                    else:
                        extracted_text += "[SUB-HEADER]"
                    extracted_text += text + "[/SUB-HEADER]\n"
                    prev_tag = "SUB-HEADER"
                else:  # paragraph
                    if prev_tag == "PARAGRAPH":
                        extracted_text = extracted_text.rstrip("[/PARAGRAPH]\n") + " "
                    else:
                        extracted_text += "[PARAGRAPH]"
                    extracted_text += text + "[/PARAGRAPH]\n"
                    prev_tag = "PARAGRAPH"

    return extracted_text

def extract_content_from_pdf(pdf_path, start_page=None, end_page=None):
    content_dict = {
        'tables': [],
        'text': []
    }

    for page_number in range(start_page, end_page + 1):
        page_layout = next(extract_pages(pdf_path, laparams=LAParams(), page_numbers=[page_number - 1]))

        # Tables extraction
        tables_content = extract_tables_from_page(pdf_path, page_number, page_layout)
        if tables_content:
            content_dict['tables'].append({
                'page': page_number,
                'content': tables_content
            })

        # Text extraction (can be enhanced further to separate headings, etc.)
        text_content = extract_refined_text(page_layout, detect_table_regions_for_page(page_layout))
        if text_content:
            content_dict['text'].append({
                'page': page_number,
                'content': text_content
            })

    return content_dict


def get_pdf_content_from_url_3(url, page_start=1, n_pages=None):
    try:
        response = requests.get(url)
        response.raise_for_status()

        with BytesIO(response.content) as pdf_data:
            temp_pdf_path = "./data/pdf_db/temp_pdf_file.pdf"
            with open(temp_pdf_path, 'wb') as temp_file:
                temp_file.write(pdf_data.read())

            end_page = page_start + n_pages - 1 if n_pages else None
            content_dict = extract_content_from_pdf(temp_pdf_path, start_page=page_start, end_page=end_page)
            return content_dict

    except requests.RequestException as e:
        print(f"Error fetching the PDF: {e}")
        return {}
    except Exception as e:
        print(f"Error processing the PDF: {e}")
        return {}

url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-and-sustainability-report-2022.pdf"
content_dict = get_pdf_content_from_url_3(url, page_start=17, n_pages=1)
print(content_dict)


{'tables': [{'page': 17, 'content': [  6%Net operating revenues $37,266 $33,014 $38,655 $ 43,004     5 Unnamed: 0  \
0     20200Operating income 10,086   8,997  10,308   10,909   NaN       2019   
3                            8,920   7,747   9,771    9,542   NaN        NaN   
5                              NaN     NaN     NaN      NaN -10.0        NaN   
6         of The Coca-Cola Company     NaN     NaN      NaN   NaN       (9%)   

    10 Unnamed: 4  
0  5.0        NaN  
3  NaN        NaN  
5  0.0        NaN  
6  NaN       2020  ,                             Unnamed: 0 Organic Revenue Growth  \
0                                  NaN            (Non-GAAP)1   
1  (in millions except per share data)                    NaN   
2                                  NaN                     20   
4                                  NaN                     15   
5                Summary of Operations                    NaN   
6                                  NaN                     10   

  Com

### Attempt 5:
Instead of extracting tables using one method, try to extract them using all available methods, and then hopefully during the embedding phase and similarity search you'll be able to find the most probable table to use.

In [23]:
def make_columns_unique(df):
    """Renaming duplicate column names."""
    cols = pd.Series(df.columns[:])
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols

def dataframe_to_prettytable(df):
    """Convert a Pandas DataFrame to a prettytable string."""
    
    # Create a new PrettyTable
    table = PrettyTable()
    
    # Handle duplicate column names
    make_columns_unique(df)

    # Add field names (column headers)
    table.field_names = df.columns.tolist()

    # Add rows
    for index, row in df.iterrows():
        table.add_row(row)

    # Return the table as a string
    return str(table)

def is_valid_table(table):
    df = table.df

    # 1. Text Density Check
    text_density = sum(len(cell) for row in df.values for cell in row) / (df.shape[0] * df.shape[1])
    if text_density < 3:  # This threshold might need tweaking
        return False

    # 2. Row and Column Count
    if df.shape[0] == 1 or df.shape[1] == 1:
        return False

    # 3. Row Uniformity
    row_lengths = [sum(len(cell) for cell in row) for row in df.values]
    length_variation = max(row_lengths) - min(row_lengths)
    if length_variation > 80:  # Again, tweak if necessary
        return False

    return True

def detect_table_regions_for_page(page_layout):
    table_regions = []

    for element in page_layout:
        if isinstance(element, LTFigure):
            bbox = element.bbox
            table_regions.append(bbox)
    return table_regions

def clean_heading(heading):
    """Clean numeric prefixes from table headings."""
    return re.sub(r'^\d+%?', '', heading)

def clean_cell_value(cell):
    """Convert cells with multiple values to NaN."""
    # Convert the cell to string for regex operations
    cell_str = str(cell)
    
    # If the cell contains more than one numeric value, return NaN
    numbers = re.findall(r'\d+%?', cell_str)
    if len(numbers) > 1:
        return pd.NA  # You can use np.nan or any other NaN representation

    return cell_str.strip()

def post_process_dataframe(df):
    """
    Post-processes the extracted dataframe to handle common anomalies.
    """
    # Dynamic thresholds based on the overall sparsity of the DataFrame
    threshold_col = min(0.15, (df.count().sum() / df.size))
    threshold_row = min(0.15, (df.count(axis=1).sum() / df.size))
    
    # Remove columns that are mostly empty
    column_non_empty_ratios = df.count() / len(df)
    columns_to_drop = column_non_empty_ratios[column_non_empty_ratios < threshold_col].index
    df = df.drop(columns=columns_to_drop)

    # Remove rows that are mostly empty
    row_non_empty_ratios = df.apply(lambda row: row.count(), axis=1) / len(df.columns)
    df = df[row_non_empty_ratios > threshold_row]

    # Clean headings
    df.columns = [clean_heading(col) for col in df.columns]

    # Clean cell values
    df = df.applymap(clean_cell_value)

    return df

def extract_tables_with_custom_areas(pdf_path, page_number, page_layout):
    table_areas = ["{},{},{},{}".format(*region) for region in detect_table_regions_for_page(page_layout)]
    tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream', table_areas=table_areas, 
                              split_text=True, strip_text=' .\n', flag_size=True)

    valid_tables = [post_process_dataframe(table.df) for table in tables if is_valid_table(table)]
    return valid_tables

def extract_tables_default(pdf_path, page_number):
    tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor='stream', 
                             split_text=True, strip_text=' .\n', flag_size=True)

    valid_tables = [post_process_dataframe(table.df) for table in tables if is_valid_table(table)]
    return valid_tables

def extract_tables_with_tabula(pdf_path, page_number):
    # Check the type and value of page_number
    if not isinstance(page_number, (int, str)):
        raise TypeError(f"Unexpected type {type(page_number)} for page_number: {page_number}")
    
    # Verify the PDF path exists
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF path not found: {pdf_path}")

    try:
        tables = tabula.read_pdf(pdf_path, pages=str(page_number), multiple_tables=True)
    except Exception as e:
        raise Exception(f"Tabula read_pdf failed on page {page_number} with error: {e}")

    # We apply post-processing to each table
    processed_tables = [post_process_dataframe(table) for table in tables]
    valid_tables = [table for table in processed_tables if not table.empty]  # Filter out any tables that became empty after processing

    return valid_tables

def extract_tables_from_page(pdf_path, page_number, page_layout):
    
    def are_tables_similar(table1, table2):
        try:
            assert_frame_equal(table1, table2)
            return True
        except AssertionError:
            return False
    
    # Start with the custom areas method for Camelot
    try:
        tables_custom_areas = extract_tables_with_custom_areas(pdf_path, page_number, page_layout)
    except Exception as e:
        print(f"Error with custom areas method on page {page_number}: {e}")
        tables_custom_areas = []

    # Use default Camelot extraction
    try:
        tables_default = extract_tables_default(pdf_path, page_number)
    except Exception as e:
        print(f"Error with default method on page {page_number}: {e}")
        tables_default = []

    # Use Tabula for extraction
    try:
        tables_tabula = extract_tables_with_tabula(pdf_path, page_number)
    except Exception as e:
        print(f"Error with Tabula on page {page_number}: {e}")
        tables_tabula = []

    # Combine tables: This is a simple union. Depending on the exact issues, we might need more sophisticated merging.
    all_tables = tables_custom_areas + tables_default + tables_tabula

    # Remove redundant tables
    unique_tables = []
    for table in all_tables:
        if not any(are_tables_similar(table, unique_table) for unique_table in unique_tables):
            unique_tables.append(table)

    # Convert tables to desired format
    tables_as_pretty_strings = [dataframe_to_prettytable(table) for table in unique_tables]

    return tables_as_pretty_strings

def should_skip(text_obj, table_regions):
    for region in table_regions:
        if (text_obj.x0 >= region['x0'] and text_obj.x1 <= region['x1'] and
            text_obj.y0 >= region['y0'] and text_obj.y1 <= region['y1']):
            return True
    return False

def clean_text(text):
    # replace common artifacts with appropriate characters or remove them
    cleaned = text.replace("•", "-")
    cleaned = cleaned.replace("\x00", "")
    return cleaned

def classify_text_based_on_style(obj):
    """
    Classify text object based on font size and other attributes.
    For now, using a simple size heuristic. This might need further refinement.
    """
    if hasattr(obj, "size"):
        if obj.size > 26:  # Threshold for headers
            return "header"
        elif 20 < obj.size <= 26:  # Threshold for sub-headers or titles
            return "sub-header"
        else:
            return "paragraph"
    return "unknown"

def classify_text_based_on_content(obj):
    """Classify text object based on content heuristics."""
    text = obj.get_text()
    
    # Heuristic checks for footnotes, typically found at the bottom of pages and smaller in size.
    if text.startswith(("*", "1", "2", "3")) and obj.y0 < 200:  # 'obj.y0 < 200' checks if the text is near the bottom
        return "footnote"
    
    # Check for high frequency of numbers; could be table or graph data.
    num_count = sum([1 for char in text if char.isdigit()])
    if num_count > len(text) * 0.5:  # More than 50% numbers
        return "table_or_graph"
    
    # Check for numbers (including potential currency signs, commas, and decimals)
    number_pattern = re.compile(r'^[-£$€]?([\d,]+(\.\d{1,2})?\%?\s*)+$')
    
    if number_pattern.match(text):
        return "number"
    
    return None

def extract_refined_text(layout, table_regions):
    extracted_text = ""
    prev_tag = None

    for obj in layout:
        if isinstance(obj, LTTextBox) or isinstance(obj, LTTextLine):
            skip = should_skip(obj, table_regions)
            
            if not skip:
                text = clean_text(obj.get_text())
                text_type_style = classify_text_based_on_style(obj)
                text_type_content = classify_text_based_on_content(obj)

                if text_type_content == "footnote":
                    if prev_tag == "FOOTNOTE":
                        extracted_text = extracted_text.rstrip("[/FOOTNOTE]\n") + " "
                    else:
                        extracted_text += "[FOOTNOTE]"
                    extracted_text += text + "[/FOOTNOTE]\n"
                    prev_tag = "FOOTNOTE"
                elif text_type_content == "table_or_graph":
                    if prev_tag == "DATA":
                        extracted_text = extracted_text.rstrip("[/DATA]\n") + "\n"
                    else:
                        extracted_text += "[DATA]"
                    extracted_text += text + "[/DATA]\n"
                    prev_tag = "DATA"    
                elif text_type_content == "number":
                    if prev_tag == "NUMBER":
                        extracted_text = extracted_text.rstrip("[/NUMBER]\n") + " "
                    else:
                        extracted_text += "[NUMBER]"
                    extracted_text += text + "[/NUMBER]\n"
                    prev_tag = "NUMBER"
                elif text_type_style == "header":
                    extracted_text += "[HEADER]" + text + "[/HEADER]\n\n"
                elif text_type_style == "sub-header":
                    if prev_tag == "SUB-HEADER":
                        extracted_text = extracted_text.rstrip("[/SUB-HEADER]\n") + " "
                    else:
                        extracted_text += "[SUB-HEADER]"
                    extracted_text += text + "[/SUB-HEADER]\n"
                    prev_tag = "SUB-HEADER"
                else:  # paragraph
                    if prev_tag == "PARAGRAPH":
                        extracted_text = extracted_text.rstrip("[/PARAGRAPH]\n") + " "
                    else:
                        extracted_text += "[PARAGRAPH]"
                    extracted_text += text + "[/PARAGRAPH]\n"
                    prev_tag = "PARAGRAPH"

    return extracted_text

def extract_content_from_pdf(pdf_path, start_page=None, end_page=None):
    content_dict = {
        'tables': [],
        'text': []
    }

    for page_number in range(start_page, end_page + 1):
        page_layout = next(extract_pages(pdf_path, laparams=LAParams(), page_numbers=[page_number - 1]))

        # Tables extraction
        tables_content = extract_tables_from_page(pdf_path, page_number, page_layout)
        if tables_content:
            content_dict['tables'].append({
                'page': page_number,
                'content': tables_content
            })

        # Text extraction (can be enhanced further to separate headings, etc.)
        text_content = extract_refined_text(page_layout, detect_table_regions_for_page(page_layout))
        if text_content:
            content_dict['text'].append({
                'page': page_number,
                'content': text_content
            })

    return content_dict

def get_pdf_content_from_url_3(url, page_start=1, n_pages=None):
    try:
        response = requests.get(url)
        response.raise_for_status()

        with BytesIO(response.content) as pdf_data:
            temp_pdf_path = "./data/pdf_db/temp_pdf_file.pdf"
            with open(temp_pdf_path, 'wb') as temp_file:
                temp_file.write(pdf_data.read())

            end_page = page_start + n_pages - 1 if n_pages else None
            content_dict = extract_content_from_pdf(temp_pdf_path, start_page=page_start, end_page=end_page)
            return content_dict

    except requests.RequestException as e:
        print(f"Error fetching the PDF: {e}")
        return {}
#     except Exception as e:
#         print(f"Error processing the PDF: {e}")
#         return {}

url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-and-sustainability-report-2022.pdf"
content_dict = get_pdf_content_from_url_3(url, page_start=17, n_pages=1)
print(content_dict)

{'tables': [{'page': 17, 'content': ['+--------------------------------+---------+---------+----------+------+------------+------+------------+\n| Net operating revenues $37,266 | $33,014 | $38,655 | $ 43,004 |      | Unnamed: 0 |  _1  | Unnamed: 4 |\n+--------------------------------+---------+---------+----------+------+------------+------+------------+\n|              <NA>              |   <NA>  |   <NA>  |   <NA>   | nan  |    2019    | <NA> |    nan     |\n|              <NA>              |   <NA>  |   <NA>  |   <NA>   | nan  |    nan     | nan  |    nan     |\n|              nan               |   nan   |   nan   |   nan    | <NA> |    nan     | <NA> |    nan     |\n|    of The Coca-Cola Company    |   nan   |   nan   |   nan    | nan  |    (9%)    | nan  |    2020    |\n+--------------------------------+---------+---------+----------+------+------------+------+------------+', '+-------------------------------------+------------------------+---------------------------------------+

### Attempt 6: 
Maybe before using the LLM we can also have a tool that tries to find the solution using Ctrl + F. That's what we try and implement below.

In [8]:
import requests

import PyPDF2

from io import BytesIO

from prettytable import PrettyTable


def search_pdf_from_url(url, query):

    # Fetch the PDF with SSL verification disabled (not recommended for production)

    response = requests.get(url, verify=False)

    # Initialize a PDF reader object using the fetched content

    pdf_reader = PyPDF2.PdfReader(BytesIO(response.content))

    # Initialize a table to store the results

    table = PrettyTable()

    table.field_names = ["Page Number", "Sentence"]

    # Loop through each page in the PDF

    for page_num in range(len(pdf_reader.pages)):

        # Get the text content of the page

        page = pdf_reader.pages[page_num]

        text = page.extract_text()

        # Split the text into sentences based on full stops

        sentences = text.split('. ')

        # Search for the query in each sentence

        for sentence in sentences:

            if query.lower() in sentence.lower():

                # Add the result to the table

                table.add_row([page_num + 1, sentence]) 

    # Check if any results were found

    if len(table._rows) == 0:

        return f"'{query}' not found in the PDF."

    else:

        return str(table)
    
# Example usage
pdf_url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-sustainability-report-2022.pdf"
query = "Summary of Operations"
result = search_pdf_from_url(pdf_url, query)
print(result)

+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Page Number |                                                                          Sentence                                                                          |
+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|      17     |                                                                                                                                                            |
|             |                                           2    R eported operating income grew 10%, declined 11%, grew 15% and                                             |
|             |  grew 6% for the years ended December 31, 2019, 2020, 2021 and 2022, respectively.3   R eported diluted earnings per sh

### Attempt 7:

Let's return back to the attempt that used selenium and see if we can make it work at scale.

In [35]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import time
import pyperclip

def extract_tables_from_pdf(url, page_numbers=None):
    # Setup for Firefox
    firefox_options = webdriver.FirefoxOptions()
    # If there are specific options you want to set for Firefox, add them here

    driver_path = "./drivers/geckodriver.exe"
    service = Service(driver_path)
    driver = webdriver.Firefox(service=service, options=firefox_options)

    driver.get(url)
    time.sleep(5)  # Waiting for the PDF to load

    if not page_numbers:
        # Get the total number of pages in the PDF
        total_pages = int(driver.find_element(By.CSS_SELECTOR, 'span#numPages').text)

        # Default to scraping all pages
        page_numbers = list(range(1, total_pages + 1))

    # Store the extracted tables
    extracted_tables = []

    for page_number in page_numbers:
        # Navigating to the specific page using the 'Next' button
        next_button = driver.find_element(By.CSS_SELECTOR, '#next')
        while True:
            current_page = int(driver.find_element(By.CSS_SELECTOR, 'input#pageNumber').get_attribute("value"))
            if current_page == page_number:
                break
            next_button.click()
            time.sleep(1)  # Wait for the page to load

        # Using JavaScript to select and copy all content within the textLayer for the current page
        js_script = """
        var range = document.createRange();
        var content = document.querySelector('div.page:nth-child({}) > div:nth-child(2)');
        range.selectNodeContents(content);
        window.getSelection().addRange(range);
        return true;
        """.format(page_number)
        success = driver.execute_script(js_script)

        if success:
            # Use JavaScript to copy the selected content
            driver.execute_script("document.execCommand('copy');")
            time.sleep(2)  # Waiting for the clipboard to populate
            page_content = pyperclip.paste()
            extracted_tables.append(page_content)  # Accumulate the content

    driver.quit()

    return extracted_tables

# Example usage
url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-sustainability-report-2022.pdf"
tables = extract_tables_from_pdf(url, page_numbers=[17, 18])
for table in tables:
    print(table)


17THE COCA-COLA COMPANY 2022 BUSINESS & SUSTAINABILITY REPORT
HOW WE OPERATEAT A GLANCEOVERVIEW INNOVATION FINANCIAL HIGHLIGHTS GOVERNANCE & MANAGEMENT PRIORITY TOPICS STAKEHOLDER ENGAGEMENT & PARTNERSHIPS
CEO MESSAGE EXECUTIVE SUMMARYCONTENTS AGRICULTUREPORTFOLIO DATA APPENDIXPACKAGING FRAMEWORKSWATER OPERATIONSCLIMATE PEOPLEOUR COMPANY
FINANCIAL HIGHLIGHTS
Note: See pages 72–74 for reconciliations of non-GAAP financial measures to our results as reported under accounting principles generally accepted
in the United States (U.S. GAAP).
1 Reported net operating revenues grew 9%, declined 11%, grew 17%
and grew 11% for the years ended December 31, 2019, 2020, 2021
and 2022, respectively.
2 Reported operating income grew 10%, declined 11%, grew 15% and
grew 6% for the years ended December 31, 2019, 2020, 2021 and
2022, respectively.
3 Reported diluted earnings per share grew 38%, declined 13%, grew
26% and declined 3% for the years ended December 31, 2019, 2020,
2021 and 2022, respectivel

In [57]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pyperclip

def extract_content_from_pdf(url, page_numbers=None, content_type="text"):
    # Setup for Firefox
    firefox_options = webdriver.FirefoxOptions()

    driver_path = "./drivers/geckodriver.exe"
    service = Service(driver_path)
    driver = webdriver.Firefox(service=service, options=firefox_options)
    wait = WebDriverWait(driver, 30)  # 30 seconds timeout

    driver.get(url)
    time.sleep(5)  # Waiting for the PDF to load

    if not page_numbers:
        # Get the total number of pages in the PDF
        total_pages = int(driver.find_element(By.CSS_SELECTOR, 'span#numPages').text)
        # Default to scraping all pages
        page_numbers = list(range(1, total_pages + 1))

    extracted_content = []

    for page_number in page_numbers:
        # Navigating to the specific page using the 'Next' button
        next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#next')))
        while True:
            current_page = int(driver.find_element(By.CSS_SELECTOR, 'input#pageNumber').get_attribute("value"))
            if current_page == page_number:
                break
            next_button.click()
            time.sleep(2)  # Give some buffer time for the change to initiate and page to load

        if content_type == "text":
            js_script = """
            var range = document.createRange();
            var content = document.querySelector('div.page:nth-child({0}) > div.textLayer');
            range.selectNodeContents(content);
            window.getSelection().addRange(range);
            return true;
            """.format(page_number)

            success = driver.execute_script(js_script)
            if success:
                # Use JavaScript to copy the selected content
                driver.execute_script("document.execCommand('copy');")
                time.sleep(2)  # Waiting for the clipboard to populate
                page_content = pyperclip.paste()
                extracted_content.append(page_content)

        elif content_type == "tables":
            js_script = """
            var rows = [];
            var spans = document.querySelectorAll('div.page:nth-child({0}) .textLayer .markedContent');
            
            function getTopValue(span) {{
                var style = span.getAttribute('style');
                var match = style.match(/top: calc\\(var\\(--scale-factor\\)\\*(.*?)px\\)/);
                return match ? parseFloat(match[1]) : null;
            }}

            spans.forEach(function(span) {{
                var topValue = getTopValue(span);
                var text = span.textContent.trim();

                if (text) {{
                    // Check if we can append to the last row
                    if (rows.length && Math.abs(rows[rows.length - 1].top - topValue) < 10) {{
                        rows[rows.length - 1].values.push(text);
                    }} else {{
                        // Or start a new row
                        rows.push({{top: topValue, values: [text]}});
                    }}
                }}
            }});

            // Convert the rows to just arrays of values
            return rows.map(row => row.values);
            """.format(page_number)

            table_data = driver.execute_script(js_script)
            extracted_content.append(table_data)

    driver.quit()
    return extracted_content

# Example usage
url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-sustainability-report-2022.pdf"
content = extract_content_from_pdf(url, page_numbers=[17, 18], content_type="tables")
for item in content:
    print(item)


JavascriptException: Message: TypeError: style is null
Stacktrace:
getTopValue@resource://pdf.js/web/viewer.html:8:29
@resource://pdf.js/web/viewer.html:13:43
@resource://pdf.js/web/viewer.html:12:19
@resource://pdf.js/web/viewer.html:30:8


Leaving the above as work and progress and instead moving to a completely different apporach.

### Attempt 8: 
Use existing tools from adobe (for instance) to convert a pdf to a docx file first and then read that into memory. Note that the order of tables, figures, and text (when everything is read back into memory) does not matter. What does matter is that we can chunck the text, and that information appearing in tables is accurate. If some information is lost in the process that's acceptable. But what can't be accepted is information corruption, and unstructured tables. 

#### Part A)
Read a docx into memory. If this works than we can look to create an automated approach for pdf conversion. 
Note that doing the conversion manually (i.e. with a human in the loop checking the converted text) may also be acceptable as poor convertion may require that we use a different pdf loader function. 

In [58]:
import docx
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from prettytable import PrettyTable
import os

def read_docx(file_path):
    """
    Read a .docx file and extract its contents into structured formats.
    
    Args:
        file_path (str): Path to the .docx file.

    Returns:
        dict: A dictionary containing:
            'text': A list of paragraphs.
            'tables': A list of pretty tables.
            'images': A list of image paths.
    """
    doc = docx.Document(file_path)
    result = {
        'text': [],
        'tables': [],
    }

    # Extract paragraphs
    for para in doc.paragraphs:
        result['text'].append(para.text)
    
    # Extract tables
    for table in doc.tables:
        pt = PrettyTable()
        for row in table.rows:
            pt.add_row([cell.text for cell in row.cells])
        result['tables'].append(pt)

    return result

file_path = "./data/docx_db/coca-cola-business-sustainability-report-2022.docx"
doc_contents = read_docx(file_path)

# Example usage:
print("\nText Extracted:")
for para in doc_contents['text']:
    print(para)

print("\nTables Extracted:")
for table in doc_contents['tables']:
    print(table)


Text Extracted:
Refresh the World. Make a Difference.

CONTENTS

	
























We build loved brands that bring joy to our consumers’ lives with beverage choices for all occasions, tastes and lifestyles. Our growth strategy is grounded in our core values and commitment to social and environmental responsibility.









 
	
	
	
	
	
	
	
 	














	







SCOPE OF THIS REPORT
This 2022 Business & Sustainability Report is The Coca-Cola Company’s fifth report to integrate overall business and sustainability performance, data and context, reflecting our continued journey toward driving sustainable business practices into our core strategy.
Except as otherwise noted, this report covers the 2022 performance of The Coca-Cola Company and the Coca-Cola system (our company and our bottling partners), as applicable.
As used in this report, the terms “material,” “materiality,” “immaterial,” “substantive,” “significant” and other similar terminology are not used, or intended to

#### Part b)
With the doc loaded properly we move to automating the conversion process using selenium. Note that this solution may need to be reworked if the adobe site changes.

In [65]:
# PDF -> DOCX

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver import Firefox
import pyautogui
import time


def convert_pdf_to_docx(pdf_filename, driver_path="./drivers/geckodriver.exe", pdf_folder_path="C:\\Users\\MD726YR\\PycharmProjects\\eyalytics\\data\\pdf_db", docx_folder_path="C:\\Users\\MD726YR\\PycharmProjects\\eyalytics\\data\\docx_db"):
    
    firefox_options = webdriver.FirefoxOptions()
    firefox_options.set_preference("browser.download.folderList", 2)
    firefox_options.set_preference("browser.download.dir", docx_folder_path)
    firefox_options.set_preference("browser.download.useDownloadDir", True)
    firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.openxmlformats-officedocument.wordprocessingml.document")

    service = Service(driver_path)
    driver = Firefox(service=service, options=firefox_options)
    wait = WebDriverWait(driver, 180)

    driver.get("https://www.adobe.com/be_en/acrobat/online/pdf-to-word.html")

    # Click the "Select a file" button
    upload_btn = wait.until(EC.element_to_be_clickable((By.ID, "lifecycle-nativebutton")))
    upload_btn.click()

    # Wait for the file selection dialog and input the file path using pyautogui
    time.sleep(2)
    pyautogui.write(f"{pdf_folder_path}\\{pdf_filename}")

    # Add a slight delay and then press 'enter' multiple times
    time.sleep(0.5)
    for _ in range(3):
        pyautogui.press('enter')
        time.sleep(0.1)

    # Wait for conversion to finish
    time.sleep(120)

    # Click the new "Download" button
    # Using a more concise selector based on given structure and assuming the button has the class "Download__downloadButton___2qFEa"
    download_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.Download__downloadButton___2qFEa')))
    download_btn.click()

    # Wait for download to finish
    time.sleep(60)

    driver.quit()
    
    # Convert the PDF filename to the expected DOCX filename
    expected_docx_filename = pdf_filename.replace('.pdf', '.docx')
    expected_docx_filepath = os.path.join(docx_folder_path, expected_docx_filename)
    
    # Check if the file exists in the expected location
    if os.path.exists(expected_docx_filepath):
        return expected_docx_filepath
    else:
        return None

if __name__ == "__main__":
    pdf_file = "coca-cola-business-and-sustainability-report-2022.pdf"
    doxc_path = convert_pdf_to_docx(pdf_file)


WebDriverException: Message: Failed to decode response from marionette


#### Part C)
Let's put everything together, and run the full script. To avoid invalid paths causing a mess we add checks into the functions to ensure the validity of paths being passed between functions.

In [22]:
import os
import time
import inspect
import requests
import pyautogui
import docx

from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.oxml.table import CT_Tbl
from prettytable import PrettyTable
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver import Firefox
from selenium import webdriver


def wait_for_file(file_path, timeout=60):
    """
    Waits for a file to appear in the specified path up to the provided timeout (in seconds).
    Returns True if the file appeared within the timeout, else returns False.
    """
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        if os.path.exists(file_path):
            return True
        time.sleep(1)  # Sleep for 1 second before checking again
    
    return False


def convert_pdf_to_docx(pdf_filename, driver_path, pdf_folder_path, docx_folder_path):
    
    firefox_options = webdriver.FirefoxOptions()
    firefox_options.set_preference("browser.download.folderList", 2)
    firefox_options.set_preference("browser.download.dir", docx_folder_path)
    firefox_options.set_preference("browser.download.useDownloadDir", True)
    firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.openxmlformats-officedocument.wordprocessingml.document")

    service = Service(driver_path)
    driver = Firefox(service=service, options=firefox_options)
    wait = WebDriverWait(driver, 180)

    driver.get("https://www.adobe.com/be_en/acrobat/online/pdf-to-word.html")

    # Click the "Select a file" button
    upload_btn = wait.until(EC.element_to_be_clickable((By.ID, "lifecycle-nativebutton")))
    upload_btn.click()

    full_pdf_path = os.path.join(pdf_folder_path, pdf_filename)
    if not os.path.exists(full_pdf_path):
        print(f"File path\n{full_pdf_path}\nis not valid.")
        return None
    
    # Wait for the file selection dialog and input the file path using pyautogui
    time.sleep(10)
    # Use the path in pyautogui
    pyautogui.typewrite(full_pdf_path)

    # Add a slight delay and then press 'enter' multiple times
    time.sleep(0.5)
    for _ in range(3):
        pyautogui.press('enter')
        time.sleep(0.1)

    # Let's wait until the "Download" button is present in the DOM (not necessarily clickable).
    # Let's wait until the "Download" button is present in the DOM (not necessarily clickable).
    time.sleep(10)

    # Check for the cookie settings notification and dismiss if present
    try:
        cookie_reject_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#onetrust-reject-all-handler")))
        if cookie_reject_btn:
            cookie_reject_btn.click()
    except Exception as e:
        print("Cookie settings notification not found or failed to click.")

    download_btn = wait.until(
        EC.element_to_be_clickable(
            (By.CSS_SELECTOR, 'button.Download__downloadButton___2qFEa')
        )
    )
    download_btn.click()
    time.sleep(10)
    driver.quit()

    # Convert the PDF filename to the expected DOCX filename
    expected_docx_filename = pdf_filename.replace('.pdf', '.docx')
    expected_docx_filepath = os.path.join(docx_folder_path, expected_docx_filename)

    # Wait for the file to appear
    if wait_for_file(expected_docx_filepath, 50):
        return expected_docx_filepath
    else:
        return None
        

def download_pdf_from_url(url, save_path):
    """
    Downloads a PDF from the given URL and saves it to the specified path.
    Returns the saved file's path if successful, otherwise returns None.
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        if os.path.exists(save_path):
            return save_path
    return None


def convert_url_pdf_to_docx(
    pdf_url, driver_path="./drivers/geckodriver.exe", 
    pdf_folder_path=None, docx_folder_path=None
):
    
    # Get the current working directory
    cwd = os.getcwd()
    
    # Set default paths if not provided
    if pdf_folder_path is None:
        pdf_folder_path = os.path.join(cwd, "..", "data", "pdf_db")
    if docx_folder_path is None:
        docx_folder_path = os.path.join(cwd, "..", "data", "docx_db")
    
    # Ensure the directories exist
    if not os.path.exists(pdf_folder_path):
        os.makedirs(pdf_folder_path)
    if not os.path.exists(docx_folder_path):
        os.makedirs(docx_folder_path)
        
    # Extract the PDF filename from the URL
    pdf_filename = pdf_url.split('/')[-1]
    pdf_save_path = os.path.join(pdf_folder_path, pdf_filename)
    
    # Download and save the PDF
    saved_pdf_path = download_pdf_from_url(pdf_url, pdf_save_path)
    if not saved_pdf_path:
        return None  # Failed to download and save the PDF
    
    # Use the saved PDF in the existing conversion function
    docx_path = convert_pdf_to_docx(pdf_filename, driver_path, pdf_folder_path, docx_folder_path)
    
    return docx_path


def read_docx(file_path):
    """
    Read a .docx file and extract its contents into structured formats.
    
    Args:
        file_path (str): Path to the .docx file.

    Returns:
        dict: A dictionary containing:
            'metadata': Document metadata.
            'text': A list of paragraphs with their styles.
            'tables': A list of table contents with headers and rows in a more readable format.
    """
    doc = docx.Document(file_path)
    result = {
        'metadata': {
            'title': doc.core_properties.title,
            'author': doc.core_properties.author,
            'created': doc.core_properties.created
        },
        'text': [],
        'tables': [],
    }

    potential_table_header = None

    # Extract paragraphs
    for index, para in enumerate(doc.paragraphs):
        style = para.style.name
        clean_text = para.text.strip()

        # Check if the next element is a table
        if index + 1 < len(doc._element.body) and isinstance(doc._element.body[index + 1], CT_Tbl):
            potential_table_header = clean_text.split()  # Splitting by whitespace for multi-column headers
            continue
        
        if clean_text:  
            result['text'].append({
                'content': clean_text,
                'style': style
            })

    # Extract tables
    for table in doc.tables:
        table_content = {'headers': potential_table_header, 'rows': []}

        for row in table.rows:
            row_content = {}
            for index, cell in enumerate(row.cells):
                cell_text = ' '.join(paragraph.text.strip() for paragraph in cell.paragraphs if paragraph.text.strip())
                
                # Using potential header if available, otherwise fallback to cell index
                header = potential_table_header[index] if potential_table_header and index < len(potential_table_header) else f"Column_{index+1}"
                
                row_content[header] = cell_text

            table_content['rows'].append(row_content)
        result['tables'].append(table_content)

        # Reset the potential table header
        potential_table_header = None

    return result


if __name__ == "__main__":
    try:
        pdf_url = "https://www.coca-colacompany.com/content/dam/company/us/en/reports/coca-cola-business-sustainability-report-2022.pdf"
        docx_path = convert_url_pdf_to_docx(pdf_url)
        
        if not docx_path:
            print("Failed to convert PDF to DOCX. Exiting...")
            exit(1)

        doc_contents = read_docx(docx_path)

        # Example usage:
        print("\nText Extracted:")
        for para in doc_contents['text']:
            print(para)

        print("\nTables Extracted:")
        for table in doc_contents['tables']:
            print(table)
    except Exception as e:
        print(f"An error occurred: {e}")


Text Extracted:
{'content': 'Refresh the World. Make a Difference.', 'style': 'Normal'}
{'content': 'CONTENTS', 'style': 'Normal'}
{'content': 'We build loved brands that bring joy to our consumers’ lives with beverage choices for all occasions, tastes and lifestyles. Our growth strategy is grounded in our core values and commitment to social and environmental responsibility.', 'style': 'Heading 6'}
{'content': 'SCOPE OF THIS REPORT', 'style': 'Normal'}
{'content': 'This 2022 Business & Sustainability Report is The Coca-Cola Company’s fifth report to integrate overall business and sustainability performance, data and context, reflecting our continued journey toward driving sustainable business practices into our core strategy.', 'style': 'Body Text'}
{'content': 'Except as otherwise noted, this report covers the 2022 performance of The Coca-Cola Company and the Coca-Cola system (our company and our bottling partners), as applicable.', 'style': 'Body Text'}
{'content': 'As used in this

{'headers': [], 'rows': [{'Column_1': 'GOAL', 'Column_2': '2022 STATUS', 'Column_3': '2022 STATUS'}, {'Column_1': 'Reduce absolute', 'Column_2': '7% decline in absolute', 'Column_3': ''}, {'Column_1': 'greenhouse gas (GHG) emissions by 25% by', 'Column_2': 'emissions since 20151', 'Column_3': '7%'}, {'Column_1': '2030, against a 2015', 'Column_2': '', 'Column_3': ''}, {'Column_1': 'baseline', 'Column_2': '', 'Column_3': ''}]}
{'headers': None, 'rows': [{'Column_1': 'AMBITION'}, {'Column_1': 'To achieve net zero emissions by 2050'}]}
{'headers': None, 'rows': [{'Column_1': '', 'Column_2': 'SENIOR LEADERSHIP', 'Column_3': 'MIDDLE MANAGEMENT', 'Column_4': 'PROFESSIONALS', 'Column_5': 'TOTAL'}, {'Column_1': 'Female', 'Column_2': '39.0%', 'Column_3': '51.3%', 'Column_4': '36.0%', 'Column_5': '44.0%'}, {'Column_1': 'Male', 'Column_2': '61.0%', 'Column_3': '48.7%', 'Column_4': '64.0%', 'Column_5': '56.0%'}]}
{'headers': None, 'rows': [{'Column_1': 'American Indian/Alaskan Native', 'Column_2':

In [16]:
docx_path

'C:\\Users\\MD726YR\\PycharmProjects\\eyalytics\\..\\data\\docx_db\\coca-cola-business-sustainability-report-2022.docx'